# Phase 5c: ColabFold Multimer - GCN4 Leucine Zipper Variants

## Objective
Validate UFoE three-term free-energy model using true dimer predictions:
1. **Hydrophobic core packing** (곤) - measured by d↔d' Cβ-Cβ distances
2. **Electrostatic registry** (건) - measured by inter-chain salt bridges
3. **Geometric constraint** (균형) - measured by crossing angle

## Three Groups
- **Group A (WT)**: Balanced hydrophobic/electrostatic - baseline
- **Group B (Gon-rich)**: Enhanced hydrophobic core (a,d → Leu/Ile)
- **Group C (Geon-rich)**: Enhanced electrostatic (e,g → Glu/Lys)

## Expected Results
| Metric | A (WT) | B (Gon↑) | C (Geon↑) |
|--------|--------|----------|----------|
| Salt bridges | 4-6 | 2-4 | ≤A or random |
| d↔d' Cβ mean | 6.0-6.5 Å | 5.5-6.0 Å | >7.0 Å |
| d↔d' Cβ std | 0.4 Å | 0.3 Å | >0.6 Å |
| Crossing angle | 18-25° | 25-35° | >40° or disordered |

---
## 1. Setup and Installation

In [ ]:
# Install ColabFold (if not already installed)
import sys
import os

# Check if running on Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Google Colab - installing ColabFold...")
    !pip install -q "colabfold[alphafold] @ git+https://github.com/sokrypton/ColabFold"
    !pip install -q py3Dmol biopython numpy pandas matplotlib
else:
    print("Running locally - ensure ColabFold is installed")
    print("Install with: pip install 'colabfold[alphafold]'")

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json

# Create output directory
output_dir = Path("phase5c_results")
output_dir.mkdir(exist_ok=True)
print(f"Output directory: {output_dir.absolute()}")

---
## 2. Define Sequences

In [ ]:
# GCN4 Leucine Zipper sequences for three groups

sequences = {
    "A_WT": {
        "description": "Wild-type GCN4 leucine zipper (balanced)",
        "chain_A": "MKQLEDKVEELLSKNYHLENEVARLKKLVGER",
        "chain_B": "MKQLEDKVEELLSKNYHLENEVARLKKLVGER",
        "expected": {
            "salt_bridges": "4-6",
            "d_cb_mean": "6.0-6.5 Å",
            "crossing_angle": "18-25°"
        }
    },
    
    "B_GonRich": {
        "description": "Hydrophobic-enhanced (a,d → Leu/Ile)",
        "chain_A": "MKQLEDKVLELLSKNYHLLNEVARLKKLVGER",  # Example: E→L, E→L mutations at a,d
        "chain_B": "MKQLEDKVLELLSKNYHLLNEVARLKKLVGER",
        "expected": {
            "salt_bridges": "2-4",
            "d_cb_mean": "5.5-6.0 Å",
            "crossing_angle": "25-35°"
        }
    },
    
    "C_GeonRich": {
        "description": "Electrostatic-enhanced (e,g → Glu/Lys)",
        "chain_A": "MKQLEDKVEELLSKNYHLENEVARLKKLVGER",  # Add more E/K at e,g positions
        "chain_B": "MKQLEDKVEELLSKNYHLENEVARLKKLVGER",
        "expected": {
            "salt_bridges": "≤A or random",
            "d_cb_mean": ">7.0 Å",
            "crossing_angle": ">40° or disordered"
        }
    }
}

# Display sequences
for group_id, data in sequences.items():
    print(f"\n{'='*60}")
    print(f"Group {group_id}: {data['description']}")
    print(f"{'='*60}")
    print(f"Chain A: {data['chain_A']}")
    print(f"Chain B: {data['chain_B']}")
    print(f"\nExpected results:")
    for key, val in data['expected'].items():
        print(f"  {key}: {val}")

### ⚠️ Note: Customize Sequences
**Please modify the sequences above based on your Phase 5b-2 confirmed sequences:**
- Group B: Replace with actual Gon-rich sequence (a,d positions → Leu/Ile)
- Group C: Replace with actual Geon-rich sequence (e,g positions → Glu/Lys)

Heptad positions:
```
Position: 1234567 1234567 1234567 1234567 12
Heptad:   abcdefg abcdefg abcdefg abcdefg ab
Sequence: MKQLEDKVEELLSKNYHLENEVARLKKLVGER
          a=core(d), e,g=surface(electrostatic)
```

---
## 3. Generate FASTA Files

In [ ]:
# Generate FASTA files for ColabFold
fasta_dir = output_dir / "fasta_files"
fasta_dir.mkdir(exist_ok=True)

for group_id, data in sequences.items():
    fasta_path = fasta_dir / f"{group_id}.fasta"
    
    with open(fasta_path, 'w') as f:
        f.write(f">{group_id}_ChainA\n")
        f.write(f"{data['chain_A']}\n")
        f.write(f">{group_id}_ChainB\n")
        f.write(f"{data['chain_B']}\n")
    
    print(f"Created: {fasta_path}")
    
    # Display FASTA content
    print(f"\nContent of {group_id}.fasta:")
    print(open(fasta_path).read())
    print("-" * 60)

---
## 4. Run ColabFold Multimer Predictions

In [ ]:
# ColabFold prediction parameters
prediction_params = {
    "num_models": 5,  # Generate 5 models per group
    "num_recycles": 3,  # Number of recycling iterations
    "model_type": "alphafold2_multimer_v3",  # Use latest multimer model
    "use_templates": False,  # No templates (ab initio prediction)
    "use_amber": True,  # Use AMBER relaxation
    "max_msa": "512:1024",  # MSA depth
}

print("ColabFold Prediction Parameters:")
print(json.dumps(prediction_params, indent=2))

In [ ]:
# Run ColabFold for each group
# Note: This requires ColabFold to be properly installed

import subprocess

for group_id in sequences.keys():
    fasta_file = fasta_dir / f"{group_id}.fasta"
    result_dir = output_dir / f"results_{group_id}"
    result_dir.mkdir(exist_ok=True)
    
    print(f"\n{'='*60}")
    print(f"Running ColabFold for {group_id}...")
    print(f"{'='*60}")
    
    # ColabFold command
    cmd = [
        "colabfold_batch",
        str(fasta_file),
        str(result_dir),
        f"--num-models={prediction_params['num_models']}",
        f"--num-recycle={prediction_params['num_recycles']}",
        f"--model-type={prediction_params['model_type']}",
        "--use-gpu-relax" if prediction_params['use_amber'] else "",
        "--amber" if prediction_params['use_amber'] else "",
    ]
    
    # Remove empty strings
    cmd = [x for x in cmd if x]
    
    print(f"Command: {' '.join(cmd)}")
    print(f"Output directory: {result_dir}")
    print("\nThis may take 30-60 minutes per group...\n")
    
    try:
        # Run ColabFold
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        print(f"✓ Prediction completed for {group_id}")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"✗ Error running ColabFold for {group_id}")
        print(f"Error: {e.stderr}")
    except FileNotFoundError:
        print("⚠ ColabFold not found in PATH")
        print("Please run ColabFold manually using the generated FASTA files")
        print(f"FASTA file: {fasta_file}")
        break

### Alternative: Manual ColabFold Execution

If the automated run fails, you can run ColabFold manually:

```bash
# For Group A
colabfold_batch \
  phase5c_results/fasta_files/A_WT.fasta \
  phase5c_results/results_A_WT \
  --num-models=5 \
  --num-recycle=3 \
  --model-type=alphafold2_multimer_v3 \
  --amber \
  --use-gpu-relax

# Repeat for Group B and C
```

Or use the online ColabFold notebook:
https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb

---
## 5. Analysis Functions

In [ ]:
# Analysis functions for dimer structures

from Bio.PDB import PDBParser, Selection
from Bio.PDB.DSSP import DSSP
import numpy as np
from scipy.spatial.distance import cdist

def detect_salt_bridges(structure, cutoff=4.0, inter_chain=True):
    """
    Detect salt bridges between charged residues.
    
    Args:
        structure: BioPython structure object
        cutoff: Distance cutoff in Angstroms
        inter_chain: If True, only detect inter-chain salt bridges
    
    Returns:
        List of salt bridge tuples: (res1, res2, distance)
    """
    positive = ['LYS', 'ARG']  # K, R
    negative = ['GLU', 'ASP']  # E, D
    
    salt_bridges = []
    
    # Get all atoms
    atoms = list(structure.get_atoms())
    
    for model in structure:
        chains = list(model.get_chains())
        
        for i, chain1 in enumerate(chains):
            for chain2 in (chains[i+1:] if inter_chain else [chain1]):
                for res1 in chain1:
                    if res1.resname in positive:
                        # Get NZ (Lys) or NH1/NH2 (Arg)
                        pos_atoms = [a for a in res1 if a.name in ['NZ', 'NH1', 'NH2']]
                        
                        for res2 in chain2:
                            if res2.resname in negative:
                                # Get OE1/OE2 (Glu) or OD1/OD2 (Asp)
                                neg_atoms = [a for a in res2 if a.name in ['OE1', 'OE2', 'OD1', 'OD2']]
                                
                                # Calculate distances
                                for pa in pos_atoms:
                                    for na in neg_atoms:
                                        dist = pa - na
                                        if dist < cutoff:
                                            salt_bridges.append((
                                                f"{chain1.id}:{res1.resname}{res1.id[1]}",
                                                f"{chain2.id}:{res2.resname}{res2.id[1]}",
                                                dist
                                            ))
    
    return salt_bridges


def analyze_core_packing(structure, sequence, heptad_positions='d'):
    """
    Analyze core packing by measuring Cβ-Cβ distances at heptad positions.
    
    Args:
        structure: BioPython structure object
        sequence: Amino acid sequence
        heptad_positions: Which heptad positions to analyze (e.g., 'd', 'ad')
    
    Returns:
        Dictionary with mean, std, and all distances
    """
    # Define heptad repeat (GCN4 starts at position 2)
    # Position: M K Q L E D K V E E L L S K N Y H L E N E V A R L K K L V G E R
    # Heptad:     a b c d e f g a b c d e f g a b c d e f g a b c d e f g a b c d e
    heptad_pattern = 'abcdefg'
    
    # Map sequence positions to heptad positions (starting from position 2)
    position_map = {}
    for i in range(len(sequence)):
        heptad_pos = heptad_pattern[(i - 1) % 7] if i >= 1 else '-'
        position_map[i] = heptad_pos
    
    # Find target positions
    target_positions = [i for i, hp in position_map.items() if hp in heptad_positions]
    
    # Calculate inter-chain Cβ distances
    distances = []
    
    model = structure[0]
    chains = list(model.get_chains())
    
    if len(chains) < 2:
        print("Warning: Less than 2 chains found")
        return {'mean': None, 'std': None, 'distances': []}
    
    chain_A, chain_B = chains[0], chains[1]
    
    for pos in target_positions:
        try:
            res_A = chain_A[pos + 1]  # PDB numbering typically starts at 1
            res_B = chain_B[pos + 1]
            
            # Get Cβ atoms (CA for Glycine)
            cb_A = res_A['CB'] if 'CB' in res_A else res_A['CA']
            cb_B = res_B['CB'] if 'CB' in res_B else res_B['CA']
            
            dist = cb_A - cb_B
            distances.append(dist)
        except:
            continue
    
    if distances:
        return {
            'mean': np.mean(distances),
            'std': np.std(distances),
            'min': np.min(distances),
            'max': np.max(distances),
            'distances': distances
        }
    else:
        return {'mean': None, 'std': None, 'distances': []}


def calc_crossing_angle(structure):
    """
    Calculate crossing angle between two helices.
    
    Args:
        structure: BioPython structure object
    
    Returns:
        Crossing angle in degrees
    """
    model = structure[0]
    chains = list(model.get_chains())
    
    if len(chains) < 2:
        return None
    
    # Get CA coordinates for each chain
    ca_coords = {}
    for chain in chains[:2]:
        coords = []
        for res in chain:
            if 'CA' in res:
                coords.append(res['CA'].coord)
        ca_coords[chain.id] = np.array(coords)
    
    # Fit helix axis using PCA
    def fit_helix_axis(coords):
        centered = coords - coords.mean(axis=0)
        cov = np.cov(centered.T)
        eigenvalues, eigenvectors = np.linalg.eig(cov)
        # Principal axis is eigenvector with largest eigenvalue
        axis = eigenvectors[:, eigenvalues.argmax()]
        return axis
    
    chain_ids = list(ca_coords.keys())[:2]
    axis_A = fit_helix_axis(ca_coords[chain_ids[0]])
    axis_B = fit_helix_axis(ca_coords[chain_ids[1]])
    
    # Calculate angle between axes
    cos_angle = np.dot(axis_A, axis_B) / (np.linalg.norm(axis_A) * np.linalg.norm(axis_B))
    angle = np.arccos(np.clip(cos_angle, -1.0, 1.0)) * 180 / np.pi
    
    # Crossing angle is the acute angle
    if angle > 90:
        angle = 180 - angle
    
    return angle


def analyze_dimer(pdb_file, sequence):
    """
    Comprehensive dimer analysis.
    
    Returns:
        Dictionary with all metrics
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('dimer', pdb_file)
    
    # 1. Salt bridges
    salt_bridges = detect_salt_bridges(structure, cutoff=4.0, inter_chain=True)
    
    # 2. Core packing (d positions)
    core_packing = analyze_core_packing(structure, sequence, heptad_positions='d')
    
    # 3. Crossing angle
    crossing_angle = calc_crossing_angle(structure)
    
    return {
        'salt_bridges': {
            'count': len(salt_bridges),
            'details': salt_bridges
        },
        'core_packing': core_packing,
        'crossing_angle': crossing_angle
    }

print("Analysis functions loaded successfully!")

---
## 6. Analyze Predicted Structures

In [ ]:
# Analyze all predicted structures
import glob

results = {}

for group_id, seq_data in sequences.items():
    result_dir = output_dir / f"results_{group_id}"
    
    if not result_dir.exists():
        print(f"⚠ Results not found for {group_id}: {result_dir}")
        continue
    
    print(f"\n{'='*60}")
    print(f"Analyzing {group_id}")
    print(f"{'='*60}")
    
    # Find PDB files (relaxed models preferred)
    pdb_files = list(result_dir.glob("*_relaxed_*.pdb"))
    if not pdb_files:
        pdb_files = list(result_dir.glob("*_unrelaxed_*.pdb"))
    
    if not pdb_files:
        print(f"  No PDB files found in {result_dir}")
        continue
    
    print(f"  Found {len(pdb_files)} models")
    
    group_results = []
    
    for pdb_file in sorted(pdb_files)[:5]:  # Analyze top 5 models
        print(f"\n  Analyzing: {pdb_file.name}")
        
        try:
            analysis = analyze_dimer(pdb_file, seq_data['chain_A'])
            analysis['pdb_file'] = str(pdb_file)
            group_results.append(analysis)
            
            # Print summary
            print(f"    Salt bridges: {analysis['salt_bridges']['count']}")
            if analysis['core_packing']['mean']:
                print(f"    d↔d' Cβ mean: {analysis['core_packing']['mean']:.2f} Å")
                print(f"    d↔d' Cβ std: {analysis['core_packing']['std']:.2f} Å")
            if analysis['crossing_angle']:
                print(f"    Crossing angle: {analysis['crossing_angle']:.1f}°")
        
        except Exception as e:
            print(f"    Error analyzing {pdb_file.name}: {e}")
    
    results[group_id] = group_results

print("\n" + "="*60)
print("Analysis complete!")
print("="*60)

---
## 7. Summary and Comparison

In [ ]:
# Create summary table
summary_data = []

for group_id, group_results in results.items():
    if not group_results:
        continue
    
    # Calculate averages across all models
    salt_bridge_counts = [r['salt_bridges']['count'] for r in group_results]
    core_means = [r['core_packing']['mean'] for r in group_results if r['core_packing']['mean']]
    core_stds = [r['core_packing']['std'] for r in group_results if r['core_packing']['std']]
    crossing_angles = [r['crossing_angle'] for r in group_results if r['crossing_angle']]
    
    summary_data.append({
        'Group': group_id,
        'Description': sequences[group_id]['description'],
        'Salt Bridges': f"{np.mean(salt_bridge_counts):.1f} ± {np.std(salt_bridge_counts):.1f}",
        'd↔d\' Cβ mean (Å)': f"{np.mean(core_means):.2f} ± {np.std(core_means):.2f}" if core_means else 'N/A',
        'd↔d\' Cβ std (Å)': f"{np.mean(core_stds):.2f}" if core_stds else 'N/A',
        'Crossing angle (°)': f"{np.mean(crossing_angles):.1f} ± {np.std(crossing_angles):.1f}" if crossing_angles else 'N/A',
        'N models': len(group_results)
    })

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*80)
print("SUMMARY TABLE: ColabFold Multimer Results")
print("="*80)
print(summary_df.to_string(index=False))
print("="*80)

# Save to CSV
summary_csv = output_dir / "phase5c_summary.csv"
summary_df.to_csv(summary_csv, index=False)
print(f"\nSummary saved to: {summary_csv}")

In [ ]:
# Visualization: Bar plots comparing metrics
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

groups = list(results.keys())
colors = ['#3498db', '#e74c3c', '#2ecc71']  # Blue, Red, Green

# 1. Salt bridges
salt_bridge_means = []
salt_bridge_stds = []
for group_id in groups:
    counts = [r['salt_bridges']['count'] for r in results[group_id]]
    salt_bridge_means.append(np.mean(counts))
    salt_bridge_stds.append(np.std(counts))

axes[0].bar(groups, salt_bridge_means, yerr=salt_bridge_stds, color=colors, alpha=0.7)
axes[0].set_ylabel('Number of Salt Bridges')
axes[0].set_title('Inter-chain Salt Bridges')
axes[0].grid(axis='y', alpha=0.3)

# 2. Core packing (d↔d' Cβ mean)
core_means = []
core_stds = []
for group_id in groups:
    means = [r['core_packing']['mean'] for r in results[group_id] if r['core_packing']['mean']]
    if means:
        core_means.append(np.mean(means))
        core_stds.append(np.std(means))
    else:
        core_means.append(0)
        core_stds.append(0)

axes[1].bar(groups, core_means, yerr=core_stds, color=colors, alpha=0.7)
axes[1].set_ylabel('Distance (Å)')
axes[1].set_title('d↔d\' Cβ-Cβ Distance')
axes[1].axhline(y=6.0, color='gray', linestyle='--', alpha=0.5, label='Optimal (6.0 Å)')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

# 3. Crossing angle
angle_means = []
angle_stds = []
for group_id in groups:
    angles = [r['crossing_angle'] for r in results[group_id] if r['crossing_angle']]
    if angles:
        angle_means.append(np.mean(angles))
        angle_stds.append(np.std(angles))
    else:
        angle_means.append(0)
        angle_stds.append(0)

axes[2].bar(groups, angle_means, yerr=angle_stds, color=colors, alpha=0.7)
axes[2].set_ylabel('Angle (degrees)')
axes[2].set_title('Helix Crossing Angle')
axes[2].axhspan(18, 25, color='green', alpha=0.1, label='Optimal range')
axes[2].legend()
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plot_file = output_dir / "phase5c_comparison.png"
plt.savefig(plot_file, dpi=300, bbox_inches='tight')
print(f"\nComparison plot saved to: {plot_file}")
plt.show()

---
## 8. Validation Against Expected Results

In [ ]:
# Check if results match expectations
print("\n" + "="*80)
print("VALIDATION: Results vs. Expected")
print("="*80)

validation = []

for group_id in groups:
    if group_id not in results or not results[group_id]:
        continue
    
    expected = sequences[group_id]['expected']
    
    # Calculate actual values
    salt_bridges = np.mean([r['salt_bridges']['count'] for r in results[group_id]])
    core_mean = np.mean([r['core_packing']['mean'] for r in results[group_id] 
                         if r['core_packing']['mean']])
    crossing = np.mean([r['crossing_angle'] for r in results[group_id] 
                        if r['crossing_angle']])
    
    print(f"\n{group_id}: {sequences[group_id]['description']}")
    print("-" * 60)
    print(f"  Salt bridges:")
    print(f"    Expected: {expected['salt_bridges']}")
    print(f"    Actual: {salt_bridges:.1f}")
    
    print(f"\n  d↔d' Cβ distance:")
    print(f"    Expected: {expected['d_cb_mean']}")
    print(f"    Actual: {core_mean:.2f} Å")
    
    print(f"\n  Crossing angle:")
    print(f"    Expected: {expected['crossing_angle']}")
    print(f"    Actual: {crossing:.1f}°")
    
    # Check if results match expectations
    checks = []
    
    if group_id == 'A_WT':
        checks.append(('Salt bridges 4-6', 4 <= salt_bridges <= 6))
        checks.append(('d↔d\' 6.0-6.5Å', 6.0 <= core_mean <= 6.5))
        checks.append(('Crossing 18-25°', 18 <= crossing <= 25))
    
    elif group_id == 'B_GonRich':
        checks.append(('Salt bridges 2-4', 2 <= salt_bridges <= 4))
        checks.append(('d↔d\' 5.5-6.0Å', 5.5 <= core_mean <= 6.0))
        checks.append(('Crossing 25-35°', 25 <= crossing <= 35))
    
    elif group_id == 'C_GeonRich':
        checks.append(('Salt bridges ≤A', salt_bridges <= 6))
        checks.append(('d↔d\' >7.0Å', core_mean > 7.0))
        checks.append(('Crossing >40°', crossing > 40))
    
    print(f"\n  Validation:")
    for check_name, passed in checks:
        status = "✓ PASS" if passed else "✗ FAIL"
        print(f"    {check_name}: {status}")
    
    validation.append({
        'group': group_id,
        'checks_passed': sum([c[1] for c in checks]),
        'checks_total': len(checks)
    })

print("\n" + "="*80)
print(f"Overall validation: {sum([v['checks_passed'] for v in validation])}/"
      f"{sum([v['checks_total'] for v in validation])} checks passed")
print("="*80)

---
## 9. Export Results

In [ ]:
# Export detailed results to JSON
import json

# Prepare export data
export_data = {
    'metadata': {
        'experiment': 'Phase 5c - ColabFold Multimer',
        'date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'groups': list(sequences.keys()),
        'prediction_params': prediction_params
    },
    'sequences': sequences,
    'results': {}
}

# Add results for each group
for group_id, group_results in results.items():
    export_data['results'][group_id] = [
        {
            'model': i + 1,
            'pdb_file': r['pdb_file'],
            'salt_bridges': r['salt_bridges'],
            'core_packing': {
                'mean': float(r['core_packing']['mean']) if r['core_packing']['mean'] else None,
                'std': float(r['core_packing']['std']) if r['core_packing']['std'] else None,
                'min': float(r['core_packing']['min']) if 'min' in r['core_packing'] else None,
                'max': float(r['core_packing']['max']) if 'max' in r['core_packing'] else None,
            },
            'crossing_angle': float(r['crossing_angle']) if r['crossing_angle'] else None
        }
        for i, r in enumerate(group_results)
    ]

# Save to JSON
json_file = output_dir / "phase5c_detailed_results.json"
with open(json_file, 'w') as f:
    json.dump(export_data, f, indent=2)

print(f"Detailed results exported to: {json_file}")

# Create a summary report
report_file = output_dir / "phase5c_report.txt"
with open(report_file, 'w') as f:
    f.write("="*80 + "\n")
    f.write("Phase 5c: ColabFold Multimer - Final Report\n")
    f.write("="*80 + "\n\n")
    
    f.write("Experiment Summary:\n")
    f.write(f"  Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"  Groups analyzed: {len(results)}\n")
    f.write(f"  Total models: {sum([len(r) for r in results.values()])}\n\n")
    
    f.write(summary_df.to_string(index=False))
    f.write("\n\n")
    
    f.write("Validation Summary:\n")
    for v in validation:
        f.write(f"  {v['group']}: {v['checks_passed']}/{v['checks_total']} checks passed\n")
    
    f.write("\n" + "="*80 + "\n")

print(f"Summary report saved to: {report_file}")

print("\n✓ All results exported successfully!")

---
## 10. Visualize Structures (Optional)

In [ ]:
# Visualize structures using py3Dmol (interactive)
try:
    import py3Dmol
    
    for group_id in groups:
        result_dir = output_dir / f"results_{group_id}"
        pdb_files = list(result_dir.glob("*_relaxed_rank_001*.pdb"))  # Top ranked model
        
        if not pdb_files:
            continue
        
        pdb_file = pdb_files[0]
        
        print(f"\nVisualizing {group_id}: {pdb_file.name}")
        
        with open(pdb_file) as f:
            pdb_str = f.read()
        
        view = py3Dmol.view(width=800, height=600)
        view.addModel(pdb_str, 'pdb')
        
        # Style: cartoon for helices, stick for residues
        view.setStyle({'cartoon': {'color': 'spectrum'}})
        view.addStyle({'chain': 'A'}, {'cartoon': {'color': 'blue', 'opacity': 0.8}})
        view.addStyle({'chain': 'B'}, {'cartoon': {'color': 'red', 'opacity': 0.8}})
        
        # Highlight d positions (core residues)
        # view.addStyle({'resi': '...'}, {'stick': {'colorscheme': 'greenCarbon'}})
        
        view.zoomTo()
        view.show()

except ImportError:
    print("py3Dmol not available. Install with: pip install py3Dmol")
    print("Structures can be visualized with PyMOL, ChimeraX, or other molecular viewers.")

---
## Conclusion

This notebook performs comprehensive analysis of GCN4 leucine zipper dimer predictions using ColabFold Multimer.

### Key Metrics
1. **Inter-chain salt bridges**: Validates electrostatic registry (건 principle)
2. **d↔d' Cβ-Cβ distances**: Validates hydrophobic core packing (곤 principle)
3. **Crossing angle**: Validates geometric balance

### Expected Outcomes
- **Group A (WT)**: Balanced design - optimal metrics
- **Group B (Gon-rich)**: Tight core - reduced d↔d' distance
- **Group C (Geon-rich)**: Registry failure - increased d↔d' distance, potential collapse

### Next Steps
If validation passes (especially Group C collapse):
- **Phase 6**: Apply principles to C3/C4 cluster design
- **Phase 7**: Target protein design with specific functions

---
**Generated for UFoE Framework validation - Phase 5c**